In [5]:
import pandas as pd

orders = pd.read_csv("orders.csv")
orders.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [ ]:
users = pd.read_json("users.json")
# normalize column name to match later code
users.rename(columns={"membership": "membership_type"}, inplace=True)
users.head()


,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [6]:
import sqlite3

conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

with open("restaurants.sql", "r") as f:
    sql_script = f.read()

cursor.executescript(sql_script)

restaurants = pd.read_sql("SELECT * FROM restaurants", conn)
restaurants.head()


,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [7]:
df = orders.merge(users, on="user_id", how="left")


In [8]:
df = df.merge(restaurants, on="restaurant_id", how="left")


In [30]:
# orders.csv uses DD-MM-YYYY format — parse explicitly to avoid ambiguity
df["order_date"] = pd.to_datetime(df["order_date"], format="%d-%m-%Y")


In [10]:
df["month"] = df["order_date"].dt.month
df["quarter"] = df["order_date"].dt.to_period("Q").astype(str)


In [11]:
df.to_csv("final_food_delivery_dataset.csv", index=False)


In [14]:
#Q1
df[df["membership"] == "Gold"] \
.groupby("city")["total_amount"].sum() \
.sort_values(ascending=False)


city
Chennai      1080909.79
Pune         1003012.32
Bangalore     994702.59
Hyderabad     896740.19
Name: total_amount, dtype: float64

In [15]:
#Q2
df.groupby("cuisine")["total_amount"].mean() \
.sort_values(ascending=False)


cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: total_amount, dtype: float64

In [16]:
#Q3
user_total = df.groupby("user_id")["total_amount"].sum()
(user_total > 1000).sum()


np.int64(2544)

In [17]:
#Q4
df["rating_range"] = pd.cut(
    df["rating"],
    bins=[3.0, 3.5, 4.0, 4.5, 5.0],
    labels=["3.0–3.5", "3.6–4.0", "4.1–4.5", "4.6–5.0"]
)

df.groupby("rating_range")["total_amount"].sum() \
.sort_values(ascending=False)


C:\Users\maris\AppData\Local\Temp\ipykernel_16156\229911283.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby("rating_range")["total_amount"].sum() \


rating_range
4.6–5.0    2197030.75
4.1–4.5    1960326.26
3.0–3.5    1881754.57
3.6–4.0    1717494.41
Name: total_amount, dtype: float64

In [19]:
#Q5
df[df["membership"] == "Gold"] \
.groupby("city")["total_amount"].mean() \
.sort_values(ascending=False)


city
Chennai      808.459080
Hyderabad    806.421034
Bangalore    793.223756
Pune         781.162243
Name: total_amount, dtype: float64

In [20]:
#Q6
df.groupby("cuisine").agg(
    restaurant_count=("restaurant_id", "nunique"),
    revenue=("total_amount", "sum")
).sort_values(by=["restaurant_count", "revenue"], ascending=[True, False])


,restaurant_count,revenue
cuisine,,
Chinese,120,1930504.65
Italian,126,2024203.80
Indian,126,1971412.58
Mexican,128,2085503.09


In [22]:
#Q7
round(
    (df[df["membership"] == "Gold"].shape[0] / df.shape[0]) * 100
)


50

In [25]:
#Q8
df.groupby("restaurant_id").agg(
    avg_order_value=("total_amount", "mean"),
    order_count=("order_id", "count")
).query("order_count < 20") \
.sort_values("avg_order_value", ascending=False)



,avg_order_value,order_count
restaurant_id,,
294,1040.222308,13
262,1029.473333,18
77,1029.180833,12
193,1026.306667,15
7,1002.140625,16
...,...,...
184,621.828947,19
498,596.815556,18
192,589.972857,14


In [27]:
#Q9
df.groupby(["membership", "cuisine"])["total_amount"].sum() \
.sort_values(ascending=False)



membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

In [28]:
#Q10
df.groupby("quarter")["total_amount"].sum() \
.sort_values(ascending=False)


quarter
2023Q3    2037385.10
2023Q4    2018263.66
2023Q1    1993425.14
2023Q2    1945348.72
2024Q1      17201.50
Name: total_amount, dtype: float64